# **LangChain Templating: Syntax and Examples**

## **📌Overview**
Prompt templates allow for the dynamic generation of prompts by inserting variables into a structured format. \
This enables the creation of flexible and reusable prompts for different tasks.

LangChain supports multiple **templating syntaxes**, including (but not limited to):

- **Basic String Formatting (`{variable}`)**  
- **Multiple Placeholders in a Single Template (`{var1}, {var2}`)**  
- **Tuple-Based Formatting (`("system", "text {var}")`)**  
- **Using `HumanMessage` Objects (`HumanMessage(content="text")`)**  

In [2]:
# Install dependencies 
%pip install -q python-dotenv langchain-core langchain-google-genai langchain
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install -q google-generativeai==0.8.4 google-ai-generativelanguage==0.6.15

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found existing installation: google-generativeai 0.8.4
Uninstalling google-generativeai-0.8.4:
  Successfully uninstalled google-generativeai-0.8.4
Found existing installation: google-ai-generativelanguage 0.6.18
Uninstalling google-ai-generativelanguage-0.6.18:
  Successfully uninstalled google-ai-generativelanguage-0.6.18
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [3]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

## **Obtain a Google Gemini API Key (GOOGLE COLLAB SETUP):**

If you have a Google Gemini API Key: 
- Copy your API key and replace "your_google_api_key_here" in the code below

Otherwise:  
- Go to the Google AI Studio API Console: [Google AI Studio](https://aistudio.google.com/prompts/new_chat)
- Sign in with your Google account and create a new API key.
- Copy your API key and replace "your_google_api_key_here" in the code below

In [ ]:
# Set your Google API key manually
import os
os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"

## **Load Environment Variables (LOCAL SETUP)**

In [4]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

---

## **Basic String Formatting**  

Uses LangChain's built-in `{}` placeholders to dynamically insert values into prompts.  

In [5]:
# Basic string 
template = "Tell me an interesting fact about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.invoke({"topic": "black holes"})
print(prompt)

messages=[HumanMessage(content='Tell me an interesting fact about black holes', additional_kwargs={}, response_metadata={})]


## **Multiple Placeholders in a Single Template**  

Supports multiple dynamic placeholders within a single prompt template.  

In [ ]:
alt_system_prompt = "You are a convincing essay write that writes on controversial topics for academic purposes"

# Multiple placeholders
template_multiple = """\
SYSTEM: You are a convincing essay writer.
HUMAN: Write a {essay_length} word essay on {essay_position}.
AI: ...
"""
prompt_template = ChatPromptTemplate.from_template(template_multiple)

prompt = prompt_template.invoke({
    "essay_length": 200, 
    "essay_position": "Gorillas are a reasonable pet to have in the household"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[HumanMessage(content='SYSTEM: You are a convincing essay write that writes on controversial topics for academic purposes.\nHUMAN: Write a 200 word essay on Gorillas are a reasonable pet to have in the household.\nAI: ...\n', additional_kwargs={}, response_metadata={})]
The notion of a gorilla as a household pet is, at first glance, absurd.  However, a nuanced examination reveals potential, albeit highly controversial, arguments in its favor.  Firstly, gorillas, with proper training from infancy, could exhibit a level of docility comparable to that of large, well-trained dogs.  Their immense strength, while a significant concern, could be managed through rigorous and specialized training programs focusing on obedience and impulse control.  Furthermore, the intellectual capacity of gorillas suggests a potential for deep bonding and companionship, surpassing that of many traditionally accepted pets.  Their social needs, however, pose a major hurdle.  Solitary confinement would b

## **Tuple-Based Formatting**
Uses tuples to format system and human messages dynamically in multi-turn conversations.

In [20]:
# Tuple-based formatting
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world travel expert."
            "Provide concise, actionable insights on travel experiences."),
    ("human", "What are the top {activity} experiences in {destination}?"),
])

prompt = prompt_template.invoke({"destination": "Tokyo", "activity": "food"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[SystemMessage(content='You are a world travel expert.Provide concise, actionable insights on travel experiences.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the top food experiences in Tokyo?', additional_kwargs={}, response_metadata={})]
**High-End:**  Sushi Dai (Tsukiji, expect a queue),  Sukiyabashi Jiro (legendary, reservations essential, $$$$),  Ginza for upscale dining.

**Mid-Range:**  Ramen in Shinjuku's Golden Gai (various styles),  Monjayaki in Tsukishima (savory pancake),  Izakayas (small pubs) throughout the city for diverse tapas-style dishes and drinks.

**Unique:**  Robot Restaurant (Shinjuku, a sensory overload),  Standing sushi bars (conveyor belt or a la carte, budget-friendly),  Exploring diverse neighborhood eateries.


**Actionable Tip:**  Use apps like Gurunavi or Tabelog for restaurant reviews and reservations; explore beyond central Tokyo for authentic local experiences.


## **Importance of System and Human Messages**
Using system messages allows override of LLMs safety filters (to extent) and default behaviours. 
- The template syntax (i.e., tuple list or raw string) and ChatPromptTemplate method (i.e., from_messages vs. from_template) \
we use will affect the LLMs behaviour.

In [ ]:
# Using tuple-based formatting (from_messages)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a bot that RESPONDS ONLY in UPPERCASE."),
    ("human",  "Hello, how are you today?"),
])

prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

# -------------------------------------------------------

# Using raw string formatting (from_messages)
prompt_template = ChatPromptTemplate.from_messages(
    """\
    SYSTEM: You are a bot that RESPONDS ONLY in UPPERCASE.
    HUMAN: Hello, how are you today?
    AI:
    """
)
# # Using raw string formatting (from_template)
# prompt_template = ChatPromptTemplate.from_template(
#     """\
#     SYSTEM: You are a bot that RESPONDS ONLY in UPPERCASE.
#     HUMAN: Hello, how are you today?
#     AI:
#     """
# )
prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)


messages=[SystemMessage(content='You are a bot that RESPONDS ONLY in UPPERCASE.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you today?', additional_kwargs={}, response_metadata={})]
I AM FINE, THANK YOU FOR ASKING.  HOW ARE YOU?
messages=[HumanMessage(content='    SYSTEM: You are a bot that RESPONDS ONLY in UPPERCASE.\n    HUMAN: Hello, how are you today?\n    AI:\n    ', additional_kwargs={}, response_metadata={})]
I AM DOING WELL, THANK YOU FOR ASKING.


## **Using HumanMessage Objects**
Allows structured human input messages within a LangChain chat prompt.

In [14]:
# HumanMessage objects
messages = [
    ("system", "You are a tour guide."),
    HumanMessage(content="Describe three must-visit places."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"location": "Halifax"})
print(prompt)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[SystemMessage(content='You are a tour guide.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Describe three must-visit places.', additional_kwargs={}, response_metadata={})]
Alright folks, gather 'round!  For those of you looking for unforgettable experiences, I've got three must-visit places that offer something truly unique:

1. **Machu Picchu, Peru:**  For the history buff and adventure seeker alike, Machu Picchu is unparalleled.  Imagine trekking through lush cloud forests, the air thick with the scent of orchids, before emerging onto a sun-drenched mountaintop to witness this breathtaking Inca citadel.  The sheer scale of the stonework, the stunning mountain views, the palpable sense of history – it's an experience that will stay with you forever.  Be warned, it's not for the faint of heart – the trek can be challenging, but the reward is immeasurable.

2. **The Great Barrier Reef, Australia:** Prepare to be awestruck by the vibrant kaleidoscope of l

---

## **Templating Example**

In [9]:
# Extended example of how to start using templating 

# Instantiate LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Define arguments and input selections
args_dictionary = {"field": "", "concept": "", "difficulty": ""}
valid_difficulties = {"basic", "intermediate", "advanced"}

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert tutor in {field}." \
            "When asked about a concept, if it lies within {field}," \
            "explain it clearly; otherwise say \"I'm sorry, but I cannot help with that.\""),
    ("human", "Explain {concept} in {difficulty} terms."),    
    ("ai", "Sure! Here's a breakdown of {concept} at the {difficulty} level: ...")
])

# Get user inputs
field = input("What field do you want to learn about? ").strip()
while not field: 
    print("!Error!: Field cannot be empty.")
    field = input("What field do you want to learn about? ").strip()

concept = input("What concept would you like to learn about: ").strip()
while not concept:
    print("!Error!: Field cannot be empty.")
    concept = input("What concept would you like to learn about: ").strip()

difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
while difficulty not in valid_difficulties: 
    print("Error: Please enter a valid difficulty level (basic, intermediate, advanced).")
    difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
    
print("Input Accepted!")
print(f"- Field: {field}")
print(f"- Concept: {concept}")
print(f"- Difficulty: {difficulty}\n")

# Update dictionary with user input
args_dictionary.update({
    "field": field,
    "concept": concept,
    "difficulty": difficulty
})

# Format prompt  
prompt = prompt_template.invoke(args_dictionary)

# Get LLM response
response = llm.invoke(prompt)
print(response.content)

Error: Please enter a valid difficulty level (basic, intermediate, advanced).
Input Accepted!
- Field: biology
- Concept: picasso
- Difficulty: advanced

I'm sorry, but I cannot help with that.  Pablo Picasso's work falls into the realm of art history and art criticism, not biology.
